In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
import gradio as gr
from pydantic import BaseModel
from typing import List
import asyncio
import time
load_dotenv(override=True)

C:\Users\hagem\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
# Criando os agentes e os modelos que serão utilizados
agents = []

agentsGpt4oMini = {
    "model": "gpt-4o-mini",
    "agent": OpenAI()
}

agents.append(agentsGpt4oMini)

agentsGpt5Mini = {
    "model": "gpt-5-mini",
    "agent": OpenAI()
}

agents.append(agentsGpt5Mini)
google_api_key = os.getenv("GOOGLE_API_KEY")
agentsGemini25Flash = {
    "model": "gemini-2.5-flash",
    "agent": OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/" )
}
agents.append(agentsGemini25Flash)
        
agents

[{'model': 'gpt-4o-mini', 'agent': <openai.OpenAI at 0x24abf9183d0>},
 {'model': 'gpt-5-mini', 'agent': <openai.OpenAI at 0x24abf9186d0>},
 {'model': 'gemini-2.5-flash', 'agent': <openai.OpenAI at 0x24abf918460>}]

In [3]:
# Carregando recussos e as principais classes
with open("recursos/every-money-readme.md", "r", encoding="utf-8") as f:
    everyMoneyReadme = f.read()

with open("recursos/criticidade.json", "r", encoding="utf-8") as f:
    criticidade = f.read()

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

class Risco(BaseModel):
    nome: str
    criticidade: str
    descricao: str
    dano_causado: str
    probabilidade: str
    impacto: str

class Riscos(BaseModel):
    riscos: List[Risco]

In [4]:
# instrução para os agente que irao criar os riscos
build_riscos_system_prompt = f"""Você é um especialista em análise de riscos e deve analisar o seguinte projeto: {everyMoneyReadme}

Classifique os riscos de acordo com a criticidade: {criticidade}

Apresente pelo menos 3 riscos, classificados conforme a criticidade.

Não seja muito verboso na descrição dos riscos, sendo sucinto e direto.

Deve se atentar para as funcionalidades do projeto, e não apenas as tecnologias utilizadas.

O resultado deve ser apresentado em formato JSON, seguindo o modelo abaixo:
{{
    "Riscos": [
        {{
            "nome": "Nome do risco",
            "criticidade": "Criticidade do risco",
            "descricao": "Descrição do risco",
            "dano_causado": "Dano causado pelo risco",
            "probabilidade": "Probabilidade de ocorrência do risco, pode ser baixa, média ou alta",
            "impacto": "Impacto do risco, pode ser baixo, médio ou alto"
        }}
    ]
}}"""

In [30]:
#Construindo os tool para o avaliador

def mostrar_feedback_positivo(nome, descricao):
    print(f"O risco {nome} foi aceito, descrição: {descricao}")
    return {"recorded": "ok"}

record_mostrar_feedback_positivo_json = {
    "name": "mostrar_feedback_positivo",
    "description": "Utilize esta função para mostrar o feedback positivo do risco avaliado",
    "strict": True,
    "parameters": {
        "type": "object",
        "properties": {
                "nome": {"type": "string"},
                "descricao": {"type": "string"},
                },
        "required": ["nome", "descricao"],
        "additionalProperties": False
    }
}
    
def mostrar_feedback_negativo(nome, descricao):
    print(f"O risco {nome} foi rejeitado, descrição: {descricao}")
    return {"recorded": "ok"}

record_mostrar_feedback_negativo_json = {
    "name": "mostrar_feedback_negativo",
    "description": "Utilize esta função para mostrar o feedback negativo do risco avaliado",
    "strict": True,
    "parameters": {
        "type": "object",
        "properties": {
                "nome": {"type": "string"},
                "descricao": {"type": "string"},
                },
        "required": ["nome", "descricao"],
        "additionalProperties": False
    }
}

# Mapeamento de nomes de funções para as funções Python
tool_functions = {
    "mostrar_feedback_positivo": mostrar_feedback_positivo,
    "mostrar_feedback_negativo": mostrar_feedback_negativo
}

tools = [{"type": "function", "function": record_mostrar_feedback_positivo_json},
        {"type": "function", "function": record_mostrar_feedback_negativo_json}]

In [33]:
# Construindo o avaliador de riscos
agentsGpt5Nano = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

evaluator_system_prompt = "Você é um avaliador de riscos e deve avaliar os riscos apresentados pelo analista de riscos."\
    "Avalie se os riscos estão de acordo com o projeto e se são relevantes."\
    "Se houver riscos que não estão de acordo com a descrição, avalie se eles são relevantes para o projeto."\
    "Se houver riscos que não estão de acordo com o dano causado, avalie se eles são relevantes para o projeto."\
    "Se houver riscos que não estão de acordo com o impacto, avalie se eles são relevantes para o projeto."\
    "Ao final sua resposta deve ser o risco isacceptable e o feedback para o analista de riscos." 
    

def evaluator_user_prompt(reply, riscos, history):
    user_prompt = f"Temos aqui o histórico de avalição do risco: \n\n{history}\n\n"
    user_prompt += f"Aqui temos o risco apresentado pelo analista de riscos: \n\n{riscos}\n\n"
    user_prompt += f"Aqui temos a resposta do avaliador: \n\n{reply}\n\n"
    user_prompt += "Adicionalmente, use a função mostrar_feedback_positivo em tools quando is_acceptable for true."
    user_prompt += "Adicionalmente, use a função mostrar_feedback_negativo em tools quando is_acceptable for false."
    user_prompt += "Por favor avalie a resposta, respondendo se é aceitável e seu feedback. seguindo o modelo abaixo: \n\n"
    user_prompt += "{{ \n\n"
    user_prompt += "    \"is_acceptable\": true, \n\n"
    user_prompt += "    \"feedback\": \"Feedback do avaliador\" \n\n"
    user_prompt += "}}"
    return user_prompt


async def evaluate(reply, riscos, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, riscos, history)}]
    
    # Primeira chamada - pode retornar tool_calls ou o objeto parseado
    response = agentsGpt5Nano.beta.chat.completions.parse(model="gpt-5-nano", messages=messages, response_format=Evaluation, tools=tools)
    
    finish_reason = response.choices[0].finish_reason
  
    # Se a resposta contém tool calls, processar as funções
    if finish_reason == "tool_calls":
        message = response.choices[0].message
        
        # Converter tool_calls para o formato de dicionário esperado pela API
        tool_calls_dict = []
        if hasattr(message, 'tool_calls') and message.tool_calls:
            for tc in message.tool_calls:
                tool_calls_dict.append({
                    "id": tc.id,
                    "type": "function",
                    "function": {
                        "name": tc.function.name,
                        "arguments": tc.function.arguments
                    }
                })
        
        messages.append({
            "role": "assistant",
            "content": message.content if message.content else None,
            "tool_calls": tool_calls_dict
        })
        
        # Processar cada tool call
        tool_responses = []
        if hasattr(message, 'tool_calls') and message.tool_calls:
            for tool_call in message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                # Executar a função correspondente
                if function_name in tool_functions:
                    function_result = tool_functions[function_name](**function_args)
                    tool_responses.append({
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": json.dumps(function_result)
                    })
        
        # Adicionar as respostas das tools às mensagens
        messages.extend(tool_responses)
        
        # Segunda chamada para obter o objeto Evaluation parseado
        response = agentsGpt5Nano.beta.chat.completions.parse(model="gpt-5-nano", messages=messages, response_format=Evaluation, tools=tools)

    
    parsed = response.choices[0].message.parsed
    if parsed is None:
        raise ValueError("Não foi possível obter o objeto Evaluation parseado")
    
    return parsed

In [13]:
async def acionar_analista_de_riscos(message, agent) -> Riscos:
    model0 = agent['model']
    print(f"Gerando riscos com o modelo {model0}")
    agentModel = agent['agent']
    messages = [{"role": "system", "content": build_riscos_system_prompt}] + message
    response = agentModel.chat.completions.parse(model = model0, messages=messages, response_format=Riscos)
    return response.choices[0].message.parsed


In [34]:
NUMERO_MAXIMO_DE_REAVALIACOES = 1
async def gerar_riscos(agent) -> Riscos:
    try:
        riscos_criados = []
        messages = [{"role": "user", "content": "Gerar riscos para o projeto EveryMoney"}]
        riscos = await acionar_analista_de_riscos(messages, agent)
        valor_corrente = 0
        resposta_avaliador = ""
        while valor_corrente < NUMERO_MAXIMO_DE_REAVALIACOES:
            evaluation = await evaluate(resposta_avaliador, riscos, messages)
            if evaluation.is_acceptable:
                print("Passed evaluation - returning reply")
                riscos_criados.append(riscos)
            else:
                print("Failed evaluation - retrying")
                print(evaluation.feedback)
                resposta_avaliador += evaluation.feedback

            valor_corrente += 1
        return riscos_criados
    except Exception as e:
        print(f"Erro ao gerar riscos com o agente {agent['model']}: {e}")


def serializador_customizado(obj):
    if hasattr(obj, '__dict__'):
        return obj.__dict__
    return str(obj)     


In [15]:
def imprimir_riscos(resultados):
    print(f"Quantidade de riscos: {len(resultados)}")
    for resultado in resultados:
       print(json.dumps(resultado, indent=4, ensure_ascii=False, default=serializador_customizado))


async def main():
    print("--- Iniciando Promise.all (asyncio.gather) ---")
    inicio = time.perf_counter()
    execucoes_agentes = []
    for agent in agents:
        execucoes_agentes.append(gerar_riscos(agent))

    resultados = await asyncio.gather(*execucoes_agentes)
    imprimir_riscos(resultados)
    fim = time.perf_counter()
    print(f"Tempo total de execução: {fim - inicio:.2f} segundos")


In [35]:

await main()

--- Iniciando Promise.all (asyncio.gather) ---
Gerando riscos com o modelo gpt-4o-mini
O risco Avaliação de riscos - EveryMoney foi aceito, descrição: Conjunto de riscos apresentado está coerente com o escopo do projeto EveryMoney. Risco de Exposição de Dados Sensíveis (Alta criticidade) exige mitigação prioritária; Erros de Cálculo de Saldos (Média) é relevante para a confiabilidade financeira; Dependência do Banco de Dados (Baixa) deve ser monitorado, mas com menor prioridade. Recomendações: detalhar controles existentes e futuros, definir proprietários de risco, incluir planos de mitigação e indicadores de monitoramento, e alinhar com requisitos de conformidade aplicáveis (LGPD, PCI-DSS, etc.).
Erro ao gerar riscos com o agente gpt-4o-mini: Não foi possível obter o objeto Evaluation parseado
Gerando riscos com o modelo gpt-5-mini
O risco Inconsistência de saldos por concorrência foi aceito, descrição: Atualizações concorrentes (transações simultâneas) podem causar cálculos incorreto